# Prerequisites

1. torchvision `(pip install torchvision)`
2. download datasets (MNIST, CIFAR10)

In [81]:
from torchvision import datasets
datasets.MNIST(root='./Data/MNIST', download=True)
datasets.CIFAR10(root='./Data/CIFAR10', download=True)

# Let's make CNN

## 1. Understand dataset

### 1.0 How does a data (picture in this case) look like?

In [149]:
train_dataset = datasets.MNIST(root='./Data/MNIST', train=True, download=True)
test_dataset = datasets.MNIST(root='./Data/MNIST', train=False, download=True)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

In [150]:
train_loader.dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F46500A89B0>, 5)

In [151]:
train_loader.dataset[0][0]

In [152]:
train_loader.dataset[0][1]

5

### 1.1 Dimensions

In [153]:
train_loader.dataset.train_data[0].size()

torch.Size([28, 28])

In [ ]:
train_dataset = datasets.MNIST(root="./Data/MNIST/", download=True, train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="./Data/MNIST/", download=True, train=False, transform=transforms.ToTensor())

## 2. Create CNN

## 2.1 Convolutional layer

In [154]:
tc.nn.Conv2d?

## 2.2 Maxpooling layer

In [155]:
tc.nn.MaxPool2d?

## 2.3 Fully-connected layer

In [156]:
tc.nn.Linear?

## 2.4 Construct a CNN

In [157]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

In [158]:
class CNN(tc.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = tc.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = tc.nn.Conv2d(10, 20, kernel_size=5)
        self.fc = tc.nn.Linear(320, 10)
        self.mp = tc.nn.MaxPool2d(2)
        
        self.relu = tc.nn.functional.relu
        self.log_softmax = tc.nn.functional.log_softmax
        
    def forward(self, x):
        x_size = x.size(0)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.mp(self.relu(self.conv2(x)))
        x = x.view(x_size, -1)
        x = self.fc(x)
        return self.log_softmax(x)

In [159]:
model = CNN()
criterion = tc.nn.CrossEntropyLoss()
optimizer = tc.optim.SGD(model.parameters(), lr=0.01)

In [174]:
DataLoader?

In [168]:
for epoch in range(10):
    model.train()
    for batch_idx, (train_data, label) in enumerate(train_loader):
        train_data, label = tc.autograd.Variable(train_data), tc.autograd.Variable(label)
        pred_label = model(train_data)
        loss = criterion(pred_label, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} Batch Index: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                 .format(epoch, batch_idx, batch_idx*len(train_data), len(train_loader.dataset), 100.*batch_idx/len(train_loader),
                 loss.data[0]))
#             print("Epoch: {} Batch Index: {} [{}/{} ({:.0f})]\t Loss: {:.6f%}".format(epoch, batch_idx,
#                   batch_idx*len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader),
#                                                                                       loss.data[0]))
                  
    model.eval()
    test_loss = 0
    correct = 0
    for data, label in test_loader:
        data, label = tc.autograd.Variable(data), tc.autograd.Variable(label)
        pred_label = model(data)

        test_loss += criterion(pred_label, label).data[0]
        pred = pred_label.data.max(1, keepdim=True)[1]
        correct += pred.eq(label.data.view_as(pred)).cpu().sum()
        
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average Loss: {} Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                            len(test_loader.dataset),
                                                                           100.*correct/len(test_loader.dataset)))
                  

/home/sewook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 Batch Index: 0 [0/60000 (0%)]	Loss: 0.073992
Train Epoch: 0 Batch Index: 10 [640/60000 (1%)]	Loss: 0.055408
Train Epoch: 0 Batch Index: 20 [1280/60000 (2%)]	Loss: 0.065569
Train Epoch: 0 Batch Index: 30 [1920/60000 (3%)]	Loss: 0.071338
Train Epoch: 0 Batch Index: 40 [2560/60000 (4%)]	Loss: 0.050740
Train Epoch: 0 Batch Index: 50 [3200/60000 (5%)]	Loss: 0.176382
Train Epoch: 0 Batch Index: 60 [3840/60000 (6%)]	Loss: 0.074007
Train Epoch: 0 Batch Index: 70 [4480/60000 (7%)]	Loss: 0.021161
Train Epoch: 0 Batch Index: 80 [5120/60000 (9%)]	Loss: 0.061038
Train Epoch: 0 Batch Index: 90 [5760/60000 (10%)]	Loss: 0.019188
Train Epoch: 0 Batch Index: 100 [6400/60000 (11%)]	Loss: 0.082299
Train Epoch: 0 Batch Index: 110 [7040/60000 (12%)]	Loss: 0.089001
Train Epoch: 0 Batch Index: 120 [7680/60000 (13%)]	Loss: 0.019425
Train Epoch: 0 Batch Index: 130 [8320/60000 (14%)]	Loss: 0.102488
Train Epoch: 0 Batch Index: 140 [8960/60000 (15%)]	Loss: 0.111931
Train Epoch: 0 Batch Index: 150 [9

Train Epoch: 1 Batch Index: 300 [19200/60000 (32%)]	Loss: 0.037882
Train Epoch: 1 Batch Index: 310 [19840/60000 (33%)]	Loss: 0.099971
Train Epoch: 1 Batch Index: 320 [20480/60000 (34%)]	Loss: 0.106178
Train Epoch: 1 Batch Index: 330 [21120/60000 (35%)]	Loss: 0.035487
Train Epoch: 1 Batch Index: 340 [21760/60000 (36%)]	Loss: 0.061576
Train Epoch: 1 Batch Index: 350 [22400/60000 (37%)]	Loss: 0.033796
Train Epoch: 1 Batch Index: 360 [23040/60000 (38%)]	Loss: 0.109567
Train Epoch: 1 Batch Index: 370 [23680/60000 (39%)]	Loss: 0.069570
Train Epoch: 1 Batch Index: 380 [24320/60000 (41%)]	Loss: 0.067726
Train Epoch: 1 Batch Index: 390 [24960/60000 (42%)]	Loss: 0.089732
Train Epoch: 1 Batch Index: 400 [25600/60000 (43%)]	Loss: 0.064424
Train Epoch: 1 Batch Index: 410 [26240/60000 (44%)]	Loss: 0.014079
Train Epoch: 1 Batch Index: 420 [26880/60000 (45%)]	Loss: 0.050958
Train Epoch: 1 Batch Index: 430 [27520/60000 (46%)]	Loss: 0.020010
Train Epoch: 1 Batch Index: 440 [28160/60000 (47%)]	Loss: 0.13

Train Epoch: 2 Batch Index: 580 [37120/60000 (62%)]	Loss: 0.006150
Train Epoch: 2 Batch Index: 590 [37760/60000 (63%)]	Loss: 0.046788
Train Epoch: 2 Batch Index: 600 [38400/60000 (64%)]	Loss: 0.059932
Train Epoch: 2 Batch Index: 610 [39040/60000 (65%)]	Loss: 0.233963
Train Epoch: 2 Batch Index: 620 [39680/60000 (66%)]	Loss: 0.122770
Train Epoch: 2 Batch Index: 630 [40320/60000 (67%)]	Loss: 0.054158
Train Epoch: 2 Batch Index: 640 [40960/60000 (68%)]	Loss: 0.072216
Train Epoch: 2 Batch Index: 650 [41600/60000 (69%)]	Loss: 0.090661
Train Epoch: 2 Batch Index: 660 [42240/60000 (70%)]	Loss: 0.043070
Train Epoch: 2 Batch Index: 670 [42880/60000 (71%)]	Loss: 0.073389
Train Epoch: 2 Batch Index: 680 [43520/60000 (72%)]	Loss: 0.010953
Train Epoch: 2 Batch Index: 690 [44160/60000 (74%)]	Loss: 0.031826
Train Epoch: 2 Batch Index: 700 [44800/60000 (75%)]	Loss: 0.185253
Train Epoch: 2 Batch Index: 710 [45440/60000 (76%)]	Loss: 0.066388
Train Epoch: 2 Batch Index: 720 [46080/60000 (77%)]	Loss: 0.04

Train Epoch: 3 Batch Index: 860 [55040/60000 (92%)]	Loss: 0.024642
Train Epoch: 3 Batch Index: 870 [55680/60000 (93%)]	Loss: 0.082484
Train Epoch: 3 Batch Index: 880 [56320/60000 (94%)]	Loss: 0.204650
Train Epoch: 3 Batch Index: 890 [56960/60000 (95%)]	Loss: 0.039607
Train Epoch: 3 Batch Index: 900 [57600/60000 (96%)]	Loss: 0.051760
Train Epoch: 3 Batch Index: 910 [58240/60000 (97%)]	Loss: 0.042348
Train Epoch: 3 Batch Index: 920 [58880/60000 (98%)]	Loss: 0.051875
Train Epoch: 3 Batch Index: 930 [59520/60000 (99%)]	Loss: 0.036192

Test set: Average Loss: 0.0007564697810419602 Accuracy: 9850/10000 (98%)

Train Epoch: 4 Batch Index: 0 [0/60000 (0%)]	Loss: 0.074200
Train Epoch: 4 Batch Index: 10 [640/60000 (1%)]	Loss: 0.049307
Train Epoch: 4 Batch Index: 20 [1280/60000 (2%)]	Loss: 0.098379
Train Epoch: 4 Batch Index: 30 [1920/60000 (3%)]	Loss: 0.055635
Train Epoch: 4 Batch Index: 40 [2560/60000 (4%)]	Loss: 0.020393
Train Epoch: 4 Batch Index: 50 [3200/60000 (5%)]	Loss: 0.086028
Train Epoc

Train Epoch: 5 Batch Index: 210 [13440/60000 (22%)]	Loss: 0.028004
Train Epoch: 5 Batch Index: 220 [14080/60000 (23%)]	Loss: 0.012218
Train Epoch: 5 Batch Index: 230 [14720/60000 (25%)]	Loss: 0.037370
Train Epoch: 5 Batch Index: 240 [15360/60000 (26%)]	Loss: 0.017568
Train Epoch: 5 Batch Index: 250 [16000/60000 (27%)]	Loss: 0.044001
Train Epoch: 5 Batch Index: 260 [16640/60000 (28%)]	Loss: 0.010065
Train Epoch: 5 Batch Index: 270 [17280/60000 (29%)]	Loss: 0.033497
Train Epoch: 5 Batch Index: 280 [17920/60000 (30%)]	Loss: 0.045980
Train Epoch: 5 Batch Index: 290 [18560/60000 (31%)]	Loss: 0.043558
Train Epoch: 5 Batch Index: 300 [19200/60000 (32%)]	Loss: 0.115262
Train Epoch: 5 Batch Index: 310 [19840/60000 (33%)]	Loss: 0.031403
Train Epoch: 5 Batch Index: 320 [20480/60000 (34%)]	Loss: 0.057623
Train Epoch: 5 Batch Index: 330 [21120/60000 (35%)]	Loss: 0.039380
Train Epoch: 5 Batch Index: 340 [21760/60000 (36%)]	Loss: 0.183961
Train Epoch: 5 Batch Index: 350 [22400/60000 (37%)]	Loss: 0.02

Train Epoch: 6 Batch Index: 500 [32000/60000 (53%)]	Loss: 0.021397
Train Epoch: 6 Batch Index: 510 [32640/60000 (54%)]	Loss: 0.020228
Train Epoch: 6 Batch Index: 520 [33280/60000 (55%)]	Loss: 0.033296
Train Epoch: 6 Batch Index: 530 [33920/60000 (57%)]	Loss: 0.017601
Train Epoch: 6 Batch Index: 540 [34560/60000 (58%)]	Loss: 0.017568
Train Epoch: 6 Batch Index: 550 [35200/60000 (59%)]	Loss: 0.041719
Train Epoch: 6 Batch Index: 560 [35840/60000 (60%)]	Loss: 0.091086
Train Epoch: 6 Batch Index: 570 [36480/60000 (61%)]	Loss: 0.043699
Train Epoch: 6 Batch Index: 580 [37120/60000 (62%)]	Loss: 0.049199
Train Epoch: 6 Batch Index: 590 [37760/60000 (63%)]	Loss: 0.125411
Train Epoch: 6 Batch Index: 600 [38400/60000 (64%)]	Loss: 0.104191
Train Epoch: 6 Batch Index: 610 [39040/60000 (65%)]	Loss: 0.027638
Train Epoch: 6 Batch Index: 620 [39680/60000 (66%)]	Loss: 0.090194
Train Epoch: 6 Batch Index: 630 [40320/60000 (67%)]	Loss: 0.081705
Train Epoch: 6 Batch Index: 640 [40960/60000 (68%)]	Loss: 0.04

Train Epoch: 7 Batch Index: 780 [49920/60000 (83%)]	Loss: 0.050268
Train Epoch: 7 Batch Index: 790 [50560/60000 (84%)]	Loss: 0.043918
Train Epoch: 7 Batch Index: 800 [51200/60000 (85%)]	Loss: 0.020658
Train Epoch: 7 Batch Index: 810 [51840/60000 (86%)]	Loss: 0.059619
Train Epoch: 7 Batch Index: 820 [52480/60000 (87%)]	Loss: 0.307888
Train Epoch: 7 Batch Index: 830 [53120/60000 (88%)]	Loss: 0.031148
Train Epoch: 7 Batch Index: 840 [53760/60000 (90%)]	Loss: 0.080006
Train Epoch: 7 Batch Index: 850 [54400/60000 (91%)]	Loss: 0.039264
Train Epoch: 7 Batch Index: 860 [55040/60000 (92%)]	Loss: 0.185732
Train Epoch: 7 Batch Index: 870 [55680/60000 (93%)]	Loss: 0.054011
Train Epoch: 7 Batch Index: 880 [56320/60000 (94%)]	Loss: 0.050335
Train Epoch: 7 Batch Index: 890 [56960/60000 (95%)]	Loss: 0.017484
Train Epoch: 7 Batch Index: 900 [57600/60000 (96%)]	Loss: 0.048871
Train Epoch: 7 Batch Index: 910 [58240/60000 (97%)]	Loss: 0.008729
Train Epoch: 7 Batch Index: 920 [58880/60000 (98%)]	Loss: 0.12

Train Epoch: 9 Batch Index: 120 [7680/60000 (13%)]	Loss: 0.010790
Train Epoch: 9 Batch Index: 130 [8320/60000 (14%)]	Loss: 0.009795
Train Epoch: 9 Batch Index: 140 [8960/60000 (15%)]	Loss: 0.030223
Train Epoch: 9 Batch Index: 150 [9600/60000 (16%)]	Loss: 0.025647
Train Epoch: 9 Batch Index: 160 [10240/60000 (17%)]	Loss: 0.050233
Train Epoch: 9 Batch Index: 170 [10880/60000 (18%)]	Loss: 0.033284
Train Epoch: 9 Batch Index: 180 [11520/60000 (19%)]	Loss: 0.052149
Train Epoch: 9 Batch Index: 190 [12160/60000 (20%)]	Loss: 0.027555
Train Epoch: 9 Batch Index: 200 [12800/60000 (21%)]	Loss: 0.008512
Train Epoch: 9 Batch Index: 210 [13440/60000 (22%)]	Loss: 0.014580
Train Epoch: 9 Batch Index: 220 [14080/60000 (23%)]	Loss: 0.062830
Train Epoch: 9 Batch Index: 230 [14720/60000 (25%)]	Loss: 0.124379
Train Epoch: 9 Batch Index: 240 [15360/60000 (26%)]	Loss: 0.183968
Train Epoch: 9 Batch Index: 250 [16000/60000 (27%)]	Loss: 0.026135
Train Epoch: 9 Batch Index: 260 [16640/60000 (28%)]	Loss: 0.008321

16